length = n_layers - 2# Klasyfikacja białaczek - sieć neuronowa

## Importy

In [12]:
from itertools import product
import warnings

import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RepeatedStratifiedKFold

## Ładowanie danych
`X` to macierz danych wejściwowych, `y` to wektor oczekiwanych diagnoz.

In [13]:
def load_dataset(filename):
    data = pd.read_csv(filename)
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    return X, y
X, y = load_dataset("data.csv")

## Selekcja cech

Ranking cech za pomocą współczynnika korelacji Pearsona $\chi^2$.

In [14]:
def rank(X, y):
    best_selector = SelectKBest(score_func=chi2, k=len(X.columns))
    fit = best_selector.fit(X, y)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(X.columns)
    ranking = pd.concat([dfcolumns, dfscores], axis=1)
    ranking.columns = ["symptom", "score"]
    return ranking.sort_values(by=["score"], ascending=False)
feature_ranking = rank(X, y)
feature_ranking_names = feature_ranking["symptom"].to_list()
feature_ranking

,symptom,score
3,bleed_p,112.952047
16,marrow,61.046859
18,limf,55.064338
1,anem,52.168346
13,rbc,50.004024
17,marrow_c,41.962909
11,eyeball,19.597693
14,plt,14.883925
5,sternum,11.920742
9,testes,11.062851


## Klasyfikacja

Kodowanie dyskretnych zmiennych jako wektory binarne. Nasze zmienne mają przypisane wartości liczbowe mimo, że nie ma między innymi liniowej zależności. To znaczy krwawienie w miejscu oznaczonym jako 1 nie jest "bliższe" krwawieniu oznaczonemu jako 2 niż temu oznaczonemu jako 4. Aby mieć pewność, że sieć nie podchwyci nieistniejących zależności tego typu zmienne koduje się jako wektory binarne.

In [15]:
def encode_inputs(X):
    ohe = OneHotEncoder(handle_unknown="ignore")
    return ohe.fit_transform(X)

Funckja do trenowania sieci korzystająca z RepeatedStratifiedKFold, oraz spełniająca wymagania projektowe co do testowania sieci.

In [16]:
def classify_kfold(X, y, hidden_layer_size, num_iteration, momentum):
    scores = []
    
    mlp = MLPClassifier(
        hidden_layer_sizes=(hidden_layer_size,),
        solver="sgd",
        max_iter= num_iteration,
        momentum=momentum,
        nesterovs_momentum=True
    )

    rkf = RepeatedStratifiedKFold(2, 5, random_state=42)
    
    for index_train, index_test in rkf.split(X, y):
        X_train, X_test = X[index_train], X[index_test]
        y_train, y_test = y[index_train], y[index_test]

        mlp.fit(X_train, y_train)
        score = mlp.score(X_test, y_test)
        scores.append(score)

    return sum(scores) / len(scores)

## Testowanie

Testujemy sieć korzystając z funkcji `classify_kfold` dla zadeklarowanych wartości parametrów: wielkość wartswy ukrytej, wartość momentum, ilość cech.

In [ ]:
hidden_layer_sizes = [100, 200, 300]
momentums = [0, 1]
feature_numbers = range(1, 21, 1)
num_iteration = 1000

results = pd.DataFrame(columns=["hidden_layer_size", "num_features", "momentum", "score"])

for size, features, momentum in product(hidden_layer_sizes, feature_numbers, momentums):
    X_fit = X[feature_ranking_names[:features]]
    X_encoded = encode_inputs(X_fit)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        score = classify_kfold(X_encoded, y, size, num_iteration, momentum)
    results.loc[len(results)] = (size, features, momentum, score)
    
results.to_csv("results.csv")
results

## Podsumowanie

Z badań wynika, że optymlna ilość cech to 18 a rozmiar warstwy ukrytej to 300, oraz że wykorzystanie momentum wpływa korzystanie na uzyskane wyniki. Uruchamiamy więc sieć z optymalnymi paramterami i prezentujemy podsumowanie dokładności oraz macierz konfuzji.

In [17]:
X_fit = X[feature_ranking_names[:18]]
X_encoded = encode_inputs(X_fit)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, stratify=y)

mlp = MLPClassifier(
    hidden_layer_sizes=(5 - 2, 300,),
    solver="sgd",
    max_iter=1000,
    momentum=1,
    nesterovs_momentum=True,
)
mlp.fit(X_train, y_train)
predictions = mlp.predict(X_test)
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         7
           4       0.18      0.50      0.27         4
           5       0.24      0.67      0.35         6
           6       0.00      0.00      0.00         4
           7       0.50      0.33      0.40         6
           8       0.29      0.40      0.33         5
           9       0.50      0.29      0.36         7
          10       0.00      0.00      0.00         4
          11       0.18      0.33      0.24         6
          12       0.20      0.20      0.20         5
          13       0.10      0.25      0.14         4
          14       0.23      0.43      0.30         7
          15       0.00      0.00      0.00         4
          16       0.00      0.00      0.00         4
          17       0.00      0.00      0.00         4
          18       0.00    

c:\users\wojtek\projects\leukemia-nn\venv\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
